In [29]:
cd "c:\Users\ecometto001\Documents\Personal\Tesis"

c:\Users\ecometto001\Documents\Personal\Tesis


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
plt.style.use("dark_background")
random_state=42

In [31]:
tesis = pd.read_csv("Datos/tesis_final.csv")
tesis = tesis[tesis["tpo_programa"] <= 12]
tesis["Intercept"] = 1

In [32]:
tesis_lag = tesis[["idPaciente", "Intercept", "TAS", "tpo_programa", "Adherencia", "Adherencia_Acumulada", "TAS_Media_Acumulada"]]
lagged_vars = ["Adherencia", "TAS", "Adherencia_Acumulada", "TAS_Media_Acumulada"]
for var in lagged_vars:
    for i in range(1, 4):
        tesis_lag[f"{var}_lag{i}"] = tesis_lag.groupby("idPaciente")[var].shift(i)

In [33]:
from statsmodels.discrete.discrete_model import Logit
fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_Acumulada_lag2",  "TAS_lag1", "TAS_Media_Acumulada_lag2"]
lr = Logit(tesis_lag.dropna()["Adherencia"], tesis_lag.dropna()[fixed_effects], groups=tesis_lag["idPaciente"]).fit()
print(lr.summary())
# Las covariables para TAS no son significativas

Optimization terminated successfully.
         Current function value: 0.465016
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                 1532
Model:                          Logit   Df Residuals:                     1527
Method:                           MLE   Df Model:                            4
Date:                Thu, 09 Jun 2022   Pseudo R-squ.:                  0.1531
Time:                        19:47:28   Log-Likelihood:                -712.40
converged:                       True   LL-Null:                       -841.15
Covariance Type:            nonrobust   LLR p-value:                 1.580e-54
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     2.1239      0.840      2.530      0.011       0.

In [34]:
max_range = tesis["tpo_programa"].max()+1

# Modelos para cada tiempo con 1 variables rezagada

In [35]:
# Exogeneidad por tiempo con un solo lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(1,max_range):
    tesis_aux = tesis_lag[tesis_lag["tpo_programa"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "TAS_lag1"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if lr.pvalues["TAS_lag1"] < 0.05:
        tiempos.append(str(i))


 ************************************************** Target time: 1 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.564076
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  383
Model:                          Logit   Df Residuals:                      380
Method:                           MLE   Df Model:                            2
Date:                Thu, 09 Jun 2022   Pseudo R-squ.:                 0.06658
Time:                        19:47:28   Log-Likelihood:                -216.04
converged:                       True   LL-Null:                       -231.45
Covariance Type:            nonrobust   LLR p-value:                 2.030e-07
                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

In [36]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 


# Modelos para cada tiempo con 1 variables rezagada y promedio acumulado

In [37]:
# Exogeneidad por tiempo con un solo lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(2,max_range):
    tesis_aux = tesis_lag[tesis_lag["tpo_programa"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_Acumulada_lag2", "TAS_lag1", "TAS_Media_Acumulada_lag2"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if (lr.pvalues["TAS_lag1"] < 0.05) or (lr.pvalues["TAS_Media_Acumulada_lag2"] < 0.05):
        tiempos.append(str(i))


 ************************************************** Target time: 2 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.487979
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  383
Model:                          Logit   Df Residuals:                      378
Method:                           MLE   Df Model:                            4
Date:                Thu, 09 Jun 2022   Pseudo R-squ.:                  0.1502
Time:                        19:47:28   Log-Likelihood:                -186.90
converged:                       True   LL-Null:                       -219.92
Covariance Type:            nonrobust   LLR p-value:                 1.549e-13
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------

In [38]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 3


# Modelos para cada tiempo con 2 variables rezagadas y promedio acumulado

In [39]:
# Exogeneidad por tiempo con dos lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(3,max_range):
    tesis_aux = tesis_lag[tesis_lag["tpo_programa"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_lag2", "Adherencia_Acumulada_lag3", "TAS_lag1", "TAS_lag2", "TAS_Media_Acumulada_lag3"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if (lr.pvalues["TAS_lag1"] < 0.05) or (lr.pvalues["TAS_lag2"] < 0.05) or (lr.pvalues["TAS_Media_Acumulada_lag3"] < 0.05):
        tiempos.append(str(i))


 ************************************************** Target time: 3 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.492235
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  383
Model:                          Logit   Df Residuals:                      376
Method:                           MLE   Df Model:                            6
Date:                Thu, 09 Jun 2022   Pseudo R-squ.:                  0.1345
Time:                        19:47:28   Log-Likelihood:                -188.53
converged:                       True   LL-Null:                       -217.81
Covariance Type:            nonrobust   LLR p-value:                 8.779e-11
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------

In [40]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 6
